In [1]:
import pandas as pd
import os
import time

In [2]:
start_time = time.time()

In [3]:
pd.set_option('display.float_format', '{:,.2f}'.format)

In [4]:
directory_files = os.listdir('files')

In [5]:
def parse_file_type(token, target_ext):
    """ 
    Search the files directory for files only containing the token
    and ending with the target file extension
    token: str, required
        The file name filter
    target_ext: str, required
        The file extension filter
    """
    files = {}
    for file_name in directory_files:
        if token in file_name:
            period_split = file_name.split('.')
            file_ext = period_split[-1:][0]
            if file_ext == target_ext:
                df = pd.read_excel(os.path.join('files', file_name), 'Sheet1')
                underscore_split = file_name.split('_')
                file_year = underscore_split[2]
                df['file_year'] = file_year
                files[file_name] = df
    return files

In [6]:
bene_files = parse_file_type('Beneficiary_Summary_File', 'xlsx')

In [7]:
bene_df = pd.concat(bene_files.values())

In [8]:
bene_df.head()

,DESYNPUF_ID,BENE_BIRTH_DT,BENE_DEATH_DT,BENE_SEX_IDENT_CD,BENE_RACE_CD,BENE_ESRD_IND,SP_STATE_CODE,BENE_COUNTY_CD,BENE_HI_CVRAGE_TOT_MONS,BENE_SMI_CVRAGE_TOT_MONS,...,MEDREIMB_IP,BENRES_IP,PPPYMT_IP,MEDREIMB_OP,BENRES_OP,PPPYMT_OP,MEDREIMB_CAR,BENRES_CAR,PPPYMT_CAR,file_year
0,00013D2EFD8E45D1,19230501,nan,1,1,0,26,950,12,12,...,0,0,0,50,10,0,0,0,0,2008
1,00016F745862898F,19430101,nan,1,1,0,39,230,12,12,...,0,0,0,0,0,0,700,240,0,2008
2,0001FDD721E223DC,19360901,nan,2,1,0,39,280,12,12,...,0,0,0,0,0,0,0,0,0,2008
3,00021CA6FF03E670,19410601,nan,1,5,0,6,290,0,0,...,0,0,0,0,0,0,0,0,0,2008
4,00024B3D2352D2D0,19360801,nan,1,1,0,52,590,12,12,...,0,0,0,30,40,0,220,80,0,2008


In [9]:
diabetes = bene_df.SP_CHF.value_counts()

In [10]:
bene_df.dtypes

DESYNPUF_ID                  object
BENE_BIRTH_DT                 int64
BENE_DEATH_DT               float64
BENE_SEX_IDENT_CD             int64
BENE_RACE_CD                  int64
BENE_ESRD_IND                object
SP_STATE_CODE                 int64
BENE_COUNTY_CD                int64
BENE_HI_CVRAGE_TOT_MONS       int64
BENE_SMI_CVRAGE_TOT_MONS      int64
BENE_HMO_CVRAGE_TOT_MONS      int64
PLAN_CVRG_MOS_NUM             int64
SP_ALZHDMTA                   int64
SP_CHF                        int64
SP_CHRNKIDN                   int64
SP_CNCR                       int64
SP_COPD                       int64
SP_DEPRESSN                   int64
SP_DIABETES                   int64
SP_ISCHMCHT                   int64
SP_OSTEOPRS                   int64
SP_RA_OA                      int64
SP_STRKETIA                   int64
MEDREIMB_IP                   int64
BENRES_IP                     int64
PPPYMT_IP                     int64
MEDREIMB_OP                   int64
BENRES_OP                   

In [11]:
total_payments = bene_df.MEDREIMB_IP + bene_df.MEDREIMB_OP + bene_df.MEDREIMB_CAR

In [12]:
bene_df['total_payments'] = total_payments

In [13]:
bene_df['diabetes'] = bene_df['SP_DIABETES'].map({1: 'Diabetic', 2: 'Non-diabetic'})

In [14]:
bene_df['heart_failure'] = bene_df['SP_CHF'].map({1: 'Congestive Heart Failure', 2: 'No Heart Failure'})

In [15]:
bene_pvt = bene_df.pivot_table(index=['file_year','diabetes', 'heart_failure'], values=['BENE_HI_CVRAGE_TOT_MONS', 'total_payments'], aggfunc='sum')

In [16]:
bene_pvt = bene_pvt.reset_index()

In [17]:
def evaluate_pmpy(x):
    """
    Calculate the per member per year claims costs from the total payments and hospital coverage months
    Assume that patients are covered under both Part A and Part B
    x: data frame row, required
        Assumes that the date frame contains total coverage and payments rows
    """
    member_months = x['BENE_HI_CVRAGE_TOT_MONS']
    total_payments = x['total_payments']
    return total_payments / member_months * 12

In [18]:
bene_pvt['PMPY'] = bene_pvt.apply(evaluate_pmpy, axis=1)

In [19]:
bene_pvt.pivot_table(index=['diabetes', 'heart_failure'], columns='file_year', values='PMPY', aggfunc='mean')

file_year                                  2008      2009     2010
diabetes     heart_failure                                        
Diabetic     Congestive Heart Failure 13,642.79 11,008.22 6,174.50
             No Heart Failure          4,254.33  5,411.04 3,540.14
Non-diabetic Congestive Heart Failure  4,843.91  5,508.22 4,131.01
             No Heart Failure            818.32  1,238.14 1,094.24

In [20]:
bene_pvt.to_excel('files/bene_pvt.xlsx')

In [21]:
end_time = time.time()

In [22]:
elapsed_time = end_time - start_time

In [23]:
print(elapsed_time)

2.2560298442840576
